In [1]:
import re
import numpy as np

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></div><div class="lev1 toc-item"><a href="#Getting-the-data" data-toc-modified-id="Getting-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Getting the data</a></div><div class="lev1 toc-item"><a href="#Preprocessing" data-toc-modified-id="Preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preprocessing</a></div><div class="lev2 toc-item"><a href="#Splitting-a-sentence-of-arbitrary-length-into-fixed-input-size" data-toc-modified-id="Splitting-a-sentence-of-arbitrary-length-into-fixed-input-size-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Splitting a sentence of arbitrary length into fixed input size</a></div><div class="lev2 toc-item"><a href="#Turning-words-into-integers" data-toc-modified-id="Turning-words-into-integers-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Turning words into integers</a></div><div class="lev1 toc-item"><a href="#Constructing-the-targets-from-sentences" data-toc-modified-id="Constructing-the-targets-from-sentences-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Constructing the targets from sentences</a></div><div class="lev1 toc-item"><a href="#Constructing-the-train/dev/test-sets" data-toc-modified-id="Constructing-the-train/dev/test-sets-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Constructing the train/dev/test sets</a></div><div class="lev1 toc-item"><a href="#Ideas-for-the-architecture" data-toc-modified-id="Ideas-for-the-architecture-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Ideas for the architecture</a></div>

# Introduction
I want to build a neural network that takes a danish sentence as an input and suggests if and where commas should be placed in the sentence.

# Getting the data
It should be possible to get a corpus of danish sentence somewhere on the internet. I have to be careful about the source, since several different comma rules can be used, and I don't want to confuse the network with a mixture of the different rules.

My raw data should be a set of sentence like the following:

In [2]:
example_1 = "NEW YORK – Mens mediernes bevågenhed i denne uge var rettet mod Houston, \
        hvor stormen Harvey og de efterfølgende oversvømmelser har kostet mindst \
        39 livet, var der fire nye og potentielt væsentlige udviklinger i den løbende \
        efterforskning af forbindelserne mellem Rusland og Donald Trumps præsidentkampagnestab."

In [3]:
example_2 = "Af folk der ikke kan komme væk fra deres egen tragedie, fordi amerikanske krigsskibe spærrer vejen."

# Preprocessing
The pre-cprocessing will include a couple of steps.
1. Taking the full sentences and cutting them into pre-defined lengths
1. Turning the words into integer representations (using the top-n words)

## Splitting a sentence of arbitrary length into fixed input size
There will be quite a few decisions from the pre-processing that will probably have to be optimized like hyper-parameters later. For now I have just guessed on some values that I think will be decent.

In [4]:
def get_word_scopes(sentence):
    word_scopes = []
    word_scope = 10
    step_size = 2
    off_set = 5
    words = sentence.split()
    num_words = len(words)

    st= 0
    nd = st + word_scope
    while nd<num_words+step_size:
        temp_sentence = ' '.join(words[st:nd])
        st = st + step_size
        nd = nd + step_size
        word_scopes.append(temp_sentence)
    return word_scopes

In [5]:
get_word_scopes(example_1)

['NEW YORK – Mens mediernes bevågenhed i denne uge var',
 '– Mens mediernes bevågenhed i denne uge var rettet mod',
 'mediernes bevågenhed i denne uge var rettet mod Houston, hvor',
 'i denne uge var rettet mod Houston, hvor stormen Harvey',
 'uge var rettet mod Houston, hvor stormen Harvey og de',
 'rettet mod Houston, hvor stormen Harvey og de efterfølgende oversvømmelser',
 'Houston, hvor stormen Harvey og de efterfølgende oversvømmelser har kostet',
 'stormen Harvey og de efterfølgende oversvømmelser har kostet mindst 39',
 'og de efterfølgende oversvømmelser har kostet mindst 39 livet, var',
 'efterfølgende oversvømmelser har kostet mindst 39 livet, var der fire',
 'har kostet mindst 39 livet, var der fire nye og',
 'mindst 39 livet, var der fire nye og potentielt væsentlige',
 'livet, var der fire nye og potentielt væsentlige udviklinger i',
 'der fire nye og potentielt væsentlige udviklinger i den løbende',
 'nye og potentielt væsentlige udviklinger i den løbende efterforskning 

In [6]:
get_word_scopes(example_2)

['Af folk der ikke kan komme væk fra deres egen',
 'der ikke kan komme væk fra deres egen tragedie, fordi',
 'kan komme væk fra deres egen tragedie, fordi amerikanske krigsskibe',
 'væk fra deres egen tragedie, fordi amerikanske krigsskibe spærrer vejen.']

## Turning words into integers
For this I need a list of the most frequently occurring words in the danish language. If my corpus is big enough, I can construct it from there, but it should be possible to find a list somewhere on the internet.

# Constructing the targets from sentences
I'm not completely sure about the architecture I am going to use, but I think that the I will have a fixed input score (say 10 words rather than the entire sentence), and then loop over the entire sentence. I will have an output layer with a neuron for each word in the scope, which will signify whether or not the word should be followed by a comma.

It has two commas in it.

By splitting the sentence into individual words, we can find the index of the words that is followed by a comma.

In [7]:
def get_y(sentence):
    words = sentence.split()
    y = np.zeros((1, len(words)))
    for idx, word in enumerate(words):
        if ',' in word:
            y[0, idx] = 1
    return y


We have now constructed the target for the output neurons

In [8]:
word_scopes = get_word_scopes(example_2)
print(word_scopes[2])
print(get_y(word_scopes[2]))

kan komme væk fra deres egen tragedie, fordi amerikanske krigsskibe
[[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]]


# Constructing the train/dev/test sets
With the functions defined above it should be possible to construct the data I need using the following command
```python    
X = []
Y = []
def pre_process(sentences):
    for sentence in sentences:
        word_scopes = split_sentence_into_intervals(sentence)
        for word_scope in word_scopes:
            y = get_y(word_scope)
            integer_representations = get_integer_representation(word_scope)
            X.append(integer_representations)
            Y.append(y)
````

# Ideas for the architecture
I can probably use the same network that I have used to classify game tags from game descriptions. I would simply use a slightly different output layer, as described above.

Using Keras, the architecture I will try first will be something like this:

```python
model = Sequential()
model.add(Embedding(max_features=top_n_words,
                    embedding_size=128,
                    input_length=word_scope))
model.add(Dropout(0.5))
model.add(Conv1D(filters=64,
                 kernel_size=5,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(50))
model.add(Dense(10))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
```